# What does thid notebook do?  
using agglomerative clustering to find posible decision points in all notebooks  

# File mapping 
sklearn | clustering | ---: `./sklearn_dp.txt`

# Agglomerative Clustering

In [127]:
%matplotlib notebook

In [128]:
import os
from tqdm import tqdm
import numpy as np
from sklearn import cluster
import scipy.cluster.hierarchy as sch
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

In [129]:
from extract_func import process_file

In [130]:
import json
with open('./func_counter.json','r') as f:
    func_counter = json.load(f)
sklearn_counter = {k: func_counter[k] for k in func_counter if k.startswith('sklearn.')}

But what is the point to do clustering before choosing representative functions?
Why can't we just sort all the functions and choose short ones? 

In [132]:
# nb_path = '/projects/bdata/jupyter/target'
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

filter notebooks (which import sklearn)

In [133]:
sklearn_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'sklearn' in tokens:
        sklearn_notebooks.append(nb)
    






  0%|          | 0/162610 [00:00<?, ?it/s]




  0%|          | 512/162610 [00:00<00:31, 5106.48it/s]




  1%|          | 1231/162610 [00:00<00:28, 5591.68it/s]




  1%|          | 1961/162610 [00:00<00:26, 6013.38it/s]




  2%|▏         | 2479/162610 [00:00<00:27, 5721.17it/s]




  2%|▏         | 3095/162610 [00:00<00:27, 5846.09it/s]




  2%|▏         | 3790/162610 [00:00<00:25, 6138.42it/s]




  3%|▎         | 4505/162610 [00:00<00:24, 6410.36it/s]




  3%|▎         | 5209/162610 [00:00<00:23, 6585.49it/s]




  4%|▎         | 5907/162610 [00:00<00:23, 6697.79it/s]




  4%|▍         | 6651/162610 [00:01<00:22, 6904.12it/s]




  5%|▍         | 7392/162610 [00:01<00:22, 7047.57it/s]




  5%|▍         | 8092/162610 [00:01<00:22, 6936.16it/s]




  5%|▌         | 8853/162610 [00:01<00:21, 7124.42it/s]




  6%|▌         | 9603/162610 [00:01<00:21, 7230.72it/s]




  6%|▋         | 10326/162610 [00:01<00:21, 6950.31it/s]




  7%|▋         | 11055/162610 [00:01<00:21, 7044

# delete loops:
eg  
**sklearn.tree.DecisionTreeClassifier.fit.fit.fit**  
is equal to   
**sklearn.tree.DecisionTreeClassifier.fit**  
  
the only difference is that the first one is called multiple times  

In [234]:
def delete_loop(func):
    """
    delete continue loops in a function
    simplify **sklearn.tree.DecisionTreeClassifier.fit.fit.fit** 
          to **sklearn.tree.DecisionTreeClassifier.fit**
    """
    tokens = func.split('.')
    new_tokens = []
    for t in tokens:
        if new_tokens == [] or t!=new_tokens[-1]:
            new_tokens.append(t)
    return '.'.join(new_tokens)

In [235]:
no_loop_sklearn_funcs = []
for k in sklearn_counter:
    t = delete_loop(k)
    if t not in no_loop_sklearn_funcs:
        no_loop_sklearn_funcs.append(t)
no_loop_sklearn_funcs

['sklearn.feature_extraction.text.TfidfVectorizer',
 'sklearn.feature_extraction.text.TfidfVectorizer.fit',
 'sklearn.feature_extraction.text.TfidfVectorizer.transform',
 'sklearn.utils.resample',
 'sklearn.preprocessing.LabelBinarizer',
 'sklearn.preprocessing.LabelBinarizer.fit',
 'sklearn.preprocessing.LabelBinarizer.transform',
 'sklearn.model_selection.train_test_split',
 'sklearn.preprocessing.LabelBinarizer.inverse_transform',
 'sklearn.utils.shuffle',
 'sklearn.metrics.accuracy_score',
 'sklearn.linear_model.LogisticRegression.summary',
 'sklearn.feature_extraction.text.CountVectorizer',
 'sklearn.feature_extraction.text.CountVectorizer.fit',
 'sklearn.feature_extraction.text.CountVectorizer.transform',
 'sklearn.linear_model.LogisticRegression',
 'sklearn.cross_validation.cross_val_score',
 'sklearn.preprocessing.LabelEncoder',
 'sklearn.preprocessing.LabelEncoder.fit',
 'sklearn.preprocessing.LabelEncoder.transform',
 'sklearn.cross_validation.train_test_split',
 'sklearn.lin

In [239]:
func2vector = {}
vector_size = len(sklearn_notebooks)
err_files = []
for i, nb in enumerate(sklearn_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    try:
        funcs, linenos = process_file(os.path.join(nb_path, nb))
    except Exception as e:
        err_files.append(nb)
    funcs = [func for func in funcs if func in no_loop_sklearn_funcs]
    for func in funcs:
#         if not func.startswith('sklearn'):
#             continue
        if func not in func2vector:
            func2vector[func] = np.zeros(vector_size)
        func2vector[func][i] = 1

Log: 0 notebooks processed
Log: 10000 notebooks processed
Log: 20000 notebooks processed
Log: 30000 notebooks processed
Log: 40000 notebooks processed
Log: 50000 notebooks processed
Log: 60000 notebooks processed
Log: 70000 notebooks processed
Log: 80000 notebooks processed
Log: 90000 notebooks processed


In [240]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [164]:
cooccur_matrix = np.load('sklearn_cooccur_mat.npy')

In [241]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
np.save('sklearn_cooccur_mat.npy', cooccur_matrix)

In [256]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4, affinity="precomputed", linkage="average").fit(a)

In [257]:
clusters = []
n_samples = len(idx2func)
for child in model.children_:
    clusters.append([])
    for branch in child:
        if branch < n_samples:
            clusters[-1].append(idx2func[branch])
        else:
            clusters[-1]+=clusters[branch-n_samples]

In [259]:
temp_clusters = [sorted(c) for c in clusters if len(c)<=3]

In [295]:
for c in temp_clusters:
    if "sklearn.linear_model.MultiTaskLasso" in c:
        print(c)

In [286]:
not_root = []
sets = []
has_root = []

def find_root_in_cluster(cluster):
    """
    find a representative function in a cluster like:
     ['sklearn.preprocessing.MinMaxScaler',
      'sklearn.preprocessing.MinMaxScaler.fit_transform',
      'sklearn.preprocessing.MinMaxScaler.transform']
    return a list []
    """
    if len(cluster)>3:
        return []
    cluster = sorted(cluster)
    last_root = '*'
    cluster_roots = []
    for f in cluster:
        if not f.startswith(last_root):
            cluster_roots.append(f)
            last_root = f
        elif f not in has_root:
            has_root.append(f)
    cluster_roots = [r for r in cluster_roots if not any([r.startswith(root) for root in roots])]
    return cluster_roots
            

    

In [294]:
roots = []
for c in tqdm(temp_clusters):
#     print('-'*20)
#     print(c)
    roots+=find_root_in_cluster(c)
#     print(find_root_in_cluster(c))
roots = sorted(set(roots))
# {r: sklearn_counter[r] for r in roots if r.endswith(".predict")}
# [r for r in roots if sklearn_counter[r]>200]
roots






  0%|          | 0/1050 [00:00<?, ?it/s]




 94%|█████████▍| 986/1050 [00:00<00:00, 9849.16it/s]




100%|██████████| 1050/1050 [00:00<00:00, 9302.94it/s]

['sklearn.GridSearchCV',
 'sklearn.MLPClassifier',
 'sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.clone.fit',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.clone',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch.fit',
 'sklearn.cluster.Birch.predict',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.KMedoids',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.Ward',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.bicluster.SpectralCoclustering',
 'sklearn.cluster.estimate_bandwidth',
 'sklearn.cluster.hierarchical.AgglomerativeClustering',
 'sklearn.cluster.k_means',
 'sklearn.cluster.spectral_clustering',

In [299]:
for r in roots:
    print(r)

sklearn.GridSearchCV
sklearn.MLPClassifier
sklearn.base.BaseEstimator.get_params
sklearn.base.BaseEstimator.set_params
sklearn.base.clone
sklearn.base.clone.fit
sklearn.calibration.CalibratedClassifierCV
sklearn.calibration.calibration_curve
sklearn.clone
sklearn.cluster.AffinityPropagation
sklearn.cluster.AgglomerativeClustering
sklearn.cluster.Birch.fit
sklearn.cluster.Birch.predict
sklearn.cluster.DBSCAN
sklearn.cluster.FeatureAgglomeration
sklearn.cluster.KMeans
sklearn.cluster.KMedoids
sklearn.cluster.MeanShift
sklearn.cluster.MiniBatchKMeans
sklearn.cluster.SpectralClustering
sklearn.cluster.Ward
sklearn.cluster.affinity_propagation
sklearn.cluster.bicluster.SpectralBiclustering
sklearn.cluster.bicluster.SpectralCoclustering
sklearn.cluster.estimate_bandwidth
sklearn.cluster.hierarchical.AgglomerativeClustering
sklearn.cluster.k_means
sklearn.cluster.spectral_clustering
sklearn.countplot
sklearn.covariance.EllipticEnvelope.fit
sklearn.covariance.EllipticEnvelope.predict
sklearn.c

In [300]:
with open('./sklearn_dp.txt','w') as fout:
    fout.write('\n'.join(roots))

In [308]:
with open('./sklearn_dp.txt','r') as f:
    sklearn_dp = f.read().split()

In [309]:
# extract structure from sklearn_roots
sklearn_roots = ['.'.join(f.split('.')[:-1]) for f in sklearn_dp]

In [338]:
roots_counter = {}
for r in sklearn_roots:
    if r not in roots_counter:
        roots_counter[r]=0
    roots_counter[r]+=1
frequent_roots = {r: roots_counter[r] for r in roots_counter if roots_counter[r]>=10}
for r in frequent_roots:
    try:
        mod = import_module(r)
#         print(mod.__all__)
        dps = ['{}.{}'.format(r, obj) for obj in mod.__all__]
        sklearn_dp+=dps
#         print(dps)
    except Exception as e:
        print(e)
sklearn_dp = sorted(list(set(sklearn_dp)))

No module named 'sklearn.cross_validation'
module 'sklearn.metrics.pairwise' has no attribute '__all__'


In [139]:
with open('./cluster.json','r') as f:
    clusters = json.load(f)

In [177]:
v1 = func2vector["sklearn.cluster.Birch.fit"]
v2 = func2vector["sklearn.cluster.Birch"]
v3 = func2vector["sklearn.cluster.AgglomerativeClustering.labels_.astype"]
np.linalg.norm(v1- v3)

4.69041575982343